# Inisialisasi GPU untuk BERTopic

In [ ]:
!nvidia-smi

Mon May 15 14:23:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 4.28 MiB/s, done.
Resolving deltas: 100% (191/191), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.3 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.

# Import Data

In [ ]:
import os
import zipfile
from google.colab import files

files.upload()

!pip install kaggle
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!kaggle datasets download -d charunisa/chatgpt-sentiment-analysis

!unzip /content/chatgpt-sentiment-analysis.zip

Saving kaggle.json to kaggle.json
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 73% 10.0M/13.7M [00:00<00:00, 46.1MB/s]
100% 13.7M/13.7M [00:00<00:00, 59.1MB/s]
Archive:  /content/chatgpt-sentiment-analysis.zip
  inflating: file.csv                


In [ ]:
!kaggle datasets download -d sanlian/tweets-about-chatgpt-march-2023
!unzip /content/tweets-about-chatgpt-march-2023.zip

 78% 14.0M/18.0M [00:00<00:00, 143MB/s]
100% 18.0M/18.0M [00:00<00:00, 160MB/s]
Archive:  /content/tweets-about-chatgpt-march-2023.zip
  inflating: chatgpt-tweets-data-20230310-20230322-processed.csv  
  inflating: chatgpt-tweets-data-20230310-20230322.csv  


In [ ]:
!kaggle datasets download -d adarshsng/googlenewsvectors
!unzip /content/googlenewsvectors.zip

100% 1.64G/1.64G [00:17<00:00, 104MB/s]
100% 1.64G/1.64G [00:17<00:00, 99.4MB/s]
Archive:  /content/googlenewsvectors.zip
  inflating: GoogleNews-vectors-negative300.bin  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Library

In [ ]:
import pandas as pd
import tensorflow as tf

In [ ]:
!pip install keras-tuner
import tensorflow as tf
from keras_tuner import BayesianOptimization
from sklearn.model_selection import train_test_split
import re
import numpy as np

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

# Data Preparation

In [ ]:
df = pd.read_csv('file.csv')[['tweets', 'labels']]
df.head(10)

,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad
5,"Just launched ChatGPT, our new AI system which...",good
6,"As of 2 minutes ago, @OpenAI released their ne...",bad
7,ChatGPT coming out strong refusing to help me ...,good
8,#0penAl just deployed a thing I've been helpin...,good
9,Research preview of our newest model: ChatGPT\...,neutral


In [ ]:
df_predict = pd.read_csv('/content/chatgpt-tweets-data-20230310-20230322-processed.csv')[['Tweet']]
df_predict.head()

,Tweet
0,ChatGPT is another woke machine.
1,"of the Atlantic, or only near the Atla #推特账号 m..."
2,This thread is saved to your Notion database....
3,Prompt AI – ChatGPT #0018
4,Just had some interesting conversations with G...


In [ ]:
df_predict = df_predict.rename({'Tweet': 'tweets'}, axis=1)
df_predict.head()

,tweets
0,ChatGPT is another woke machine.
1,"of the Atlantic, or only near the Atla #推特账号 m..."
2,This thread is saved to your Notion database....
3,Prompt AI – ChatGPT #0018
4,Just had some interesting conversations with G...


In [ ]:
df['labels'] = df['labels'].map({'bad':0, 'good':1, 'neutral':2})
df.head(3)

,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,2
1,"Try talking with ChatGPT, our new AI system wh...",1
2,ChatGPT: Optimizing Language Models for Dialog...,2


# Data Cleaning

In [ ]:
def clean_text(tweet):

    # Convert to lower case
    tweet = tweet.lower()
    # remove unicode characters
    tweet = tweet.encode('ascii', 'ignore').decode()
    # Clean www.* or https?://*
    tweet = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    # Clean @username
    tweet = re.sub(r'@[^\s]+','',tweet)
    # Remove new line
    tweet = re.sub(r'\\n', ' ', tweet)
    # Remove additional white spaces
    tweet = re.sub(r'[\s]+', ' ', tweet)
    # Remove non-word character
    tweet = re.sub(r'[^a-z\s0-9]', '', tweet)
    # Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    #trim
    tweet = tweet.strip('\'"')

    return tweet

lemmatizer = WordNetLemmatizer()
def lemmatize(sentence):
    sw = stopwords.words('english')
    sw.append('chatgpt')
    sw.append('gpt')
    sw.append('openai')
    sw.append('ai')
    words = nltk.word_tokenize(sentence)
    words = [lemmatizer.lemmatize(word) for word in words if word not in sw]
    words = ' '.join(words)
    return words

df["tweets"] = df['tweets'].map(lambda x: clean_text(x))
df['tweets'] = df['tweets'].map(lambda x: lemmatize(x))
df = df[df['tweets'].apply(lambda x: len(x.split()) >=1)]

In [ ]:
df['tweets']

0                        optimizing language model dialogue
1         try talking new system optimized dialogue feed...
2         optimizing language model dialogue machinelear...
3         thrilled share new model optimized dialog publ...
4                       2 minute ago released new use right
                                ...                        
219289                    software project trying replicate
219290    asked write nye joke seos delivered seo make r...
219291                               disassembled dissemble
219292    2023 prediction nothing really specific trend ...
219293                                           neat stuff
Name: tweets, Length: 217529, dtype: object

In [ ]:
df['labels'].value_counts()

0    106032
1     56010
2     55487
Name: labels, dtype: int64

# Split Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['tweets'].values, df['labels'].values,
                                                    test_size=.3,
                                                    random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=.5,
                                                random_state=42)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((152270,), (32629,), (32630,), (152270,), (32629,), (32630,))

# Build Sentiment Analysis Model

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

y_train.shape, y_test.shape

((152270, 3), (32630, 3))

In [ ]:
y_val = tf.keras.utils.to_categorical(y_val)

y_val.shape

(32629, 3)

In [ ]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_val_sequences = tokenizer.texts_to_sequences(X_val)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_ready = pad_sequences(X_train_sequences, padding='post',
                              maxlen=128, truncating='post')
X_val_ready = pad_sequences(X_val_sequences, padding='post',
                             maxlen=128, truncating='post')
X_test_ready = pad_sequences(X_test_sequences, padding='post',
                             maxlen=128, truncating='post')

word_index = tokenizer.word_index

In [ ]:
def build_model(hp):
    Embedding = tf.keras.layers.Embedding
    Bidirectional = tf.keras.layers.Bidirectional
    Conv1D = tf.keras.layers.Conv1D
    LSTM = tf.keras.layers.LSTM
    Dropout = tf.keras.layers.Dropout
    Dense = tf.keras.layers.Dense
    regularizer = tf.keras.regularizers.L1L2

    emb_dim = hp.Int('embedding_dim', 32, 128, step=32)

    model = tf.keras.Sequential()
    model.add(Embedding(len(word_index)+1, emb_dim, input_length=128))

    # Add conv layer if True
    use_conv = hp.Boolean('conv')
    if use_conv:
        n_filter = hp.Int('conv_filter', 32, 128, step=32)
        kernel_size = hp.Int('conv_size', 3, 7)
        conv_l1 = hp.Float('l1_conv', 0, 1e-2)
        conv_l2 = hp.Float('l2_conv', 0, 1e-1)

        model.add(Conv1D(n_filter, kernel_size, activation='relu',
                         kernel_regularizer=regularizer(l1=conv_l1, l2=conv_l2)))

    # Add LSTM layer
    n_neuron_lstm = hp.Int('n_neuron_lstm', 16, 64, step=16)
    lstm_l1 = hp.Float('l1_lstm', 0, 1e-2)
    lstm_l2 = hp.Float('l2_lstm', 0, 1e-1)

    model.add(Bidirectional(LSTM(n_neuron_lstm, kernel_regularizer=regularizer(l1=lstm_l1, l2=lstm_l2))))

    # Add Dense Layer
    n_layer_dense = hp.Int('n_layer_dense', 1, 2)
    for i in range(n_layer_dense):
        n_neuron_dense = hp.Int(f'n_neuron_dense_{i}', 32, 256, step=64)
        dense_l1 = hp.Float(f'l1_dense_{i}', 0, 1e-2)
        dense_l2 = hp.Float(f'l2_dense_{i}', 0, 1e-1)
        use_dropout = hp.Boolean(f'dropout_{i}')

        model.add(Dense(n_neuron_dense, activation='relu',
                        kernel_regularizer=regularizer(l1=dense_l1, l2=dense_l2)))
        if use_dropout:
            model.add(Dropout(.2))

    # Add output layer
    model.add(Dense(3))

    # compile
    lr = hp.Float('lr', 1e-5, 1e-3)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt,
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

In [ ]:
tf.keras.backend.clear_session()

tuner = BayesianOptimization(build_model,
                             objective = 'val_accuracy',
                             max_trials = 50,
                             directory = 'tune',
                             project_name = 'Sentiment-FinPro',
                             overwrite = False)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_accuracy',
                                              restore_best_weights=True,
                                              verbose=1)

tuner.search(X_train_ready, y_train,
             batch_size=512, epochs=100,
             validation_data=(X_val_ready, y_val),
             callbacks=[early_stop])

Trial 15 Complete [00h 05m 20s]
val_accuracy: 0.8680025935173035

Best val_accuracy So Far: 0.8858657479286194
Total elapsed time: 00h 59m 46s

Search: Running Trial #16

Value             |Best Value So Far |Hyperparameter
32                |128               |embedding_dim
False             |True              |conv
16                |16                |n_neuron_lstm
0.0070308         |0.0046469         |l1_lstm
0.041217          |0.065388          |l2_lstm
2                 |1                 |n_layer_dense
96                |224               |n_neuron_dense_0
0.0033208         |0.0032938         |l1_dense_0
0.0057926         |0.094458          |l2_dense_0
False             |True              |dropout_0
0.00049265        |0.00070334        |lr
96                |32                |conv_filter
3                 |3                 |conv_size
0.0094671         |0                 |l1_conv
0.023148          |0                 |l2_conv
224               |None              |n_neuron_dense_

In [ ]:
tuner.results_summary(1)

Results summary
Results in tune/Sentiment-FinPro
Showing 1 best trials
Objective(name="val_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
embedding_dim: 128
conv: True
n_neuron_lstm: 16
l1_lstm: 0.00464685403393522
l2_lstm: 0.06538769777048237
n_layer_dense: 1
n_neuron_dense_0: 224
l1_dense_0: 0.003293811228284589
l2_dense_0: 0.09445758647359109
dropout_0: True
lr: 0.0007033412411911787
conv_filter: 32
conv_size: 3
l1_conv: 0.0
l2_conv: 0.0
Score: 0.8858657479286194


In [ ]:
X_train = np.concatenate([X_train, X_val])
y_train = np.concatenate([y_train, y_val])

In [ ]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_ready = pad_sequences(X_train_sequences, padding='post',
                              maxlen=128, truncating='post')
X_test_ready = pad_sequences(X_test_sequences, padding='post',
                             maxlen=128, truncating='post')

word_index = tokenizer.word_index

In [ ]:
def create_model():
    reg = tf.keras.regularizers.L1L2
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(len(word_index)+1, 128, input_length=128),
        tf.keras.layers.Conv1D(32, 3, activation='relu'),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, kernel_regularizer=reg(l1=5e-3, l2=7e-2))),
        tf.keras.layers.Dense(224, activation='relu', kernel_regularizer=reg(l1=3e-3, l2=9e-2)),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(3)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=7e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

In [ ]:
tf.keras.backend.clear_session()

model = create_model()

early_stop = tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_accuracy',
                                              restore_best_weights=True,
                                              verbose=1)

history = model.fit(X_train_ready, y_train, epochs=100, batch_size=512,
                    validation_data=(X_test_ready, y_test),
                    callbacks=[early_stop])

Epoch 1/100
362/362 [==============================] - 82s 179ms/step - loss: 4.7055 - accuracy: 0.6821 - val_loss: 0.8337 - val_accuracy: 0.8340
Epoch 2/100
362/362 [==============================] - 25s 70ms/step - loss: 0.5835 - accuracy: 0.8588 - val_loss: 0.5102 - val_accuracy: 0.8626
Epoch 3/100
362/362 [==============================] - 16s 45ms/step - loss: 0.4184 - accuracy: 0.8994 - val_loss: 0.4558 - val_accuracy: 0.8739
Epoch 4/100
362/362 [==============================] - 14s 38ms/step - loss: 0.3442 - accuracy: 0.9224 - val_loss: 0.4424 - val_accuracy: 0.8803
Epoch 5/100
362/362 [==============================] - 12s 34ms/step - loss: 0.2978 - accuracy: 0.9362 - val_loss: 0.4167 - val_accuracy: 0.8854
Epoch 6/100
362/362 [==============================] - 10s 28ms/step - loss: 0.2667 - accuracy: 0.9445 - val_loss: 0.4168 - val_accuracy: 0.8857
Epoch 7/100
362/362 [==============================] - 11s 30ms/step - loss: 0.2448 - accuracy: 0.9512 - val_loss: 0.4265 - val_a

In [ ]:
from sklearn.metrics import classification_report

y_pred = np.argmax(model.predict(X_test_ready), axis=-1)
print(classification_report(np.argmax(y_test, axis=-1), y_pred))

1020/1020 [==============================] - 7s 6ms/step
              precision    recall  f1-score   support

           0       0.94      0.93      0.93     16049
           1       0.87      0.92      0.89      8327
           2       0.80      0.78      0.79      8254

    accuracy                           0.89     32630
   macro avg       0.87      0.87      0.87     32630
weighted avg       0.89      0.89      0.89     32630



In [ ]:
model.save('/content/drive/MyDrive/TK2/sentiment_model.h5')

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/TK2/sentiment_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 128)          10460416  
                                                                 
 conv1d (Conv1D)             (None, 126, 32)           12320     
                                                                 
 bidirectional (Bidirectiona  (None, 32)               6272      
 l)                                                              
                                                                 
 dense (Dense)               (None, 224)               7392      
                                                                 
 dropout (Dropout)           (None, 224)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 675       
                                                        

# Predict data maret

In [ ]:
df_predict_clean = df_predict.copy()
df_predict_clean["tweets"] = df_predict_clean['tweets'].map(lambda x: clean_text(x))
df_predict_clean['tweets'] = df_predict_clean['tweets'].map(lambda x: lemmatize(x))
df_predict_clean = df_predict_clean[df_predict_clean['tweets'].apply(lambda x: len(x.split()) >=1)]

df_predict_clean

,tweets
0,another woke machine
1,atlantic near atla greenness th
2,thread saved notion database tag
3,prompt chatgpt0018
4,interesting conversation google bard appears c...
...,...
98754,sorry phone da blink n dyslexic message hopele...
98755,someone tell future version chat spin website ...
98756,talk aws infrastructure footprint
98757,yet human human already become language model


In [ ]:
X_predict = df_predict_clean.values.ravel()
X_predict_sequences = tokenizer.texts_to_sequences(X_predict)
X_predict_ready = pad_sequences(X_predict_sequences, padding='post',
                                maxlen=128, truncating='post')

X_predict_ready.shape

(97556, 128)

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/TK2/sentiment_model.h5')
model.summary()

In [ ]:
y_predict = tf.argmax(model.predict(X_predict_ready), axis=-1).numpy()
y_predict.shape

3049/3049 [==============================] - 20s 7ms/step


(97556,)

In [ ]:
df_predict['labels'] = y_predict
df_predict

,tweets,labels
0,ChatGPT is another woke machine.,0
1,"of the Atlantic, or only near the Atla #推特账号 m...",0
2,This thread is saved to your Notion database....,2
3,Prompt AI – ChatGPT #0018,0
4,Just had some interesting conversations with G...,1
...,...,...
98754,Sorry my phones on da blink n my dyslexic mes...,2
98755,If someone were to tell a future version of C...,0
98756,ChatGPT can talk to your AWS infrastructure fo...,0
98757,"ChatGPT is not yet a human, but some humans ha...",0


In [ ]:
df_predict.labels.value_counts()

0    48495
1    25566
2    23495
Name: labels, dtype: int64

# Topic Detection with Bertopic

In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 51.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [ ]:
from bertopic import BERTopic
### Topic Coherence
import gensim.models.keyedvectors as word2vec
from itertools import combinations
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
### Memuat Model Word2Vec
w2v_model = word2vec.KeyedVectors.load_word2vec_format("/content/GoogleNews-vectors-negative300.bin", binary=True)

### Fungsi Menghitung Coherence
def calculate_coherence( w2v_model, term_rankings):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for pair in combinations(term_rankings[topic_index], 2 ):
            if pair[0] in w2v_model.index_to_key:
                if pair[1] in w2v_model.index_to_key:
                    pair_scores.append( abs(w2v_model.similarity(pair[0], pair[1])) )
                #else:
                    #pair_scores.append(0)
            #else:
                #pair_scores.append(0)

        # get the mean for all pairs in this topic
        if pair_scores:
            topic_score = sum(pair_scores) / len(pair_scores)
            overall_coherence += topic_score

    # get the mean score across all topics
    return overall_coherence / len(term_rankings)

def get_top_words(topics_info):
    top_words = []
    for topic in topics_info.keys():
        top_words_per_topic = []
        for word, val in topics_info[topic]:
            top_words_per_topic.append(word)
        top_words.append(top_words_per_topic)

    return top_words

## GridSearch Tune simulasi 1

In [ ]:
import numpy as np
from copy import deepcopy

best_coherence = -np.inf
best_model = None
coherence_history = dict()
min_topic_size_list = [50, 100, 150, 200]
nr_topics_list = [10, 15, 20]
top_n_words_list = [5, 10]

for min_topic_size in min_topic_size_list:
    for nr_topics in nr_topics_list:
        for top_n_words in top_n_words_list:
            print(f'Running min_topic_size: {min_topic_size}, \
                    nr_topics: {nr_topics}, \
                    top_n_words: {top_n_words}')

            # Create instances of GPU-accelerated UMAP and HDBSCAN
            umap_model = UMAP(n_components=5, n_neighbors=15, metric='cosine')
            hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', prediction_data=True)

            topic_model = BERTopic(language="english", top_n_words=top_n_words,
                                   min_topic_size=min_topic_size,
                                   nr_topics=nr_topics,
                                   calculate_probabilities=True,
                                   umap_model=umap_model,
                                   hdbscan_model=hdbscan_model,
                                   verbose=True)

            topics, prob = topic_model.fit_transform(df_predict_clean['tweets'].values)
            new_topics = topic_model.reduce_outliers(df_predict_clean['tweets'].values,
                                                     topics, strategy="embeddings")
            topic_model.update_topics(df_predict_clean['tweets'].values, topics=new_topics)

            top_words = get_top_words(topic_model.get_topics())

            coherence_keys = f'{min_topic_size}, {nr_topics}, {top_n_words}'
            coherence = calculate_coherence(w2v_model, top_words)
            coherence_history[coherence_keys] = coherence
            if best_coherence < coherence:
                best_coherence = coherence
                best_model = deepcopy(topic_model)

            print(f'Current coherence score: {coherence}')
            print(f'Current best coherence score: {best_coherence}')

Running min_topic_size: 50,                     nr_topics: 10,                     top_n_words: 5


Batches:   0%|          | 0/3049 [00:00<?, ?it/s]

2023-05-15 13:14:05,324 - BERTopic - Transformed documents to Embeddings
2023-05-15 13:14:23,672 - BERTopic - Reduced dimensionality
2023-05-15 13:14:30,440 - BERTopic - Clustered reduced embeddings
2023-05-15 13:14:35,931 - BERTopic - Reduced number of topics from 823 to 10


Current coherence score: 0.17153402840241877
Current best coherence score: 0.17153402840241877
Running min_topic_size: 50,                     nr_topics: 10,                     top_n_words: 10


Batches:   0%|          | 0/3049 [00:00<?, ?it/s]

...

2023-05-15 13:51:56,865 - BERTopic - Transformed documents to Embeddings
2023-05-15 13:52:15,270 - BERTopic - Reduced dimensionality
2023-05-15 13:52:21,369 - BERTopic - Clustered reduced embeddings
2023-05-15 13:52:28,534 - BERTopic - Reduced number of topics from 849 to 20


Current coherence score: 0.18147955743281627
Current best coherence score: 0.1935972570500649
Running min_topic_size: 200,                     nr_topics: 20,                     top_n_words: 10


Batches:   0%|          | 0/3049 [00:00<?, ?it/s]

2023-05-15 13:53:41,696 - BERTopic - Transformed documents to Embeddings
2023-05-15 13:53:59,691 - BERTopic - Reduced dimensionality
2023-05-15 13:54:05,713 - BERTopic - Clustered reduced embeddings
2023-05-15 13:54:12,991 - BERTopic - Reduced number of topics from 857 to 20


Current coherence score: 0.1878000948415313
Current best coherence score: 0.1935972570500649


In [ ]:
print(coherence_history)
print(best_coherence)

{'50, 10, 5': 0.17153402840241877, '50, 10, 10': 0.16456239296825576, '50, 15, 5': 0.17076803809201452, '50, 15, 10': 0.18476646977809127, '50, 20, 5': 0.18233964843380804, '50, 20, 10': 0.18298519123999413, '100, 10, 5': 0.16968661046210892, '100, 10, 10': 0.17557215652472918, '100, 15, 5': 0.1816205666779139, '100, 15, 10': 0.18005820439592568, '100, 20, 5': 0.18739826480294233, '100, 20, 10': 0.1935972570500649, '150, 10, 5': 0.18569998811892535, '150, 10, 10': 0.17574800482579686, '150, 15, 5': 0.17623906175515144, '150, 15, 10': 0.1830220142215207, '150, 20, 5': 0.1842842692651968, '150, 20, 10': 0.18114674545049594, '200, 10, 5': 0.1705938858687188, '200, 10, 10': 0.178211038766451, '200, 15, 5': 0.1885106590564327, '200, 15, 10': 0.178138371606586, '200, 20, 5': 0.18147955743281627, '200, 20, 10': 0.1878000948415313}
0.1935972570500649


In [ ]:
best_model.save('/content/drive/MyDrive/TK2/chatgpt-topic-model')

In [ ]:
topic_model = BERTopic.load('/content/drive/MyDrive/TK2/chatgpt-topic-model')

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,0,58968,0_chat_gpt4_google_like
1,1,1789,1_congratulation_congrats_day_thanks
2,2,2148,2_math_answer_calculator_problem
3,3,3705,3_domain_business_linkedin_website
4,4,4079,4_app_code_like_game
5,5,1719,5_apology_chat_asked_response
6,6,3116,6_write_writing_story_written
7,7,1415,7_money_tax_pay_business
8,8,1317,8_javascript_code_react_api
9,9,3772,9_trust_people_dont_like


## Gridsearch Tune Simulasi 2

In [ ]:

import numpy as np
from copy import deepcopy

best_coherence = -np.inf
best_model = None
coherence_history = dict()
min_topic_size_list = [50, 150, 250]
nr_topics_list = [8, 12, 16]
top_n_words_list = [10, 15]

for min_topic_size in min_topic_size_list:
    for nr_topics in nr_topics_list:
        for top_n_words in top_n_words_list:
            print(f'Running min_topic_size: {min_topic_size}, \
                    nr_topics: {nr_topics}, \
                    top_n_words: {top_n_words}')

            # Create instances of GPU-accelerated UMAP and HDBSCAN
            umap_model = UMAP(n_components=5, n_neighbors=15, metric='cosine')
            hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', prediction_data=True)

            topic_model = BERTopic(language="english", top_n_words=top_n_words,
                                   min_topic_size=min_topic_size,
                                   nr_topics=nr_topics,
                                   calculate_probabilities=True,
                                   umap_model=umap_model,
                                   hdbscan_model=hdbscan_model,
                                   verbose=True)

            topics, prob = topic_model.fit_transform(df['Tweet'].values)
            new_topics = topic_model.reduce_outliers(df['Tweet'].values, topics, strategy="embeddings")
            topic_model.update_topics(df['Tweet'].values, topics=new_topics)

            top_words = get_top_words(topic_model.get_topics())

            coherence_keys = f'{min_topic_size}, {nr_topics}, {top_n_words}'
            coherence = calculate_coherence(w2v_model, top_words)
            coherence_history[coherence_keys] = coherence
            if best_coherence < coherence:
                best_coherence = coherence
                best_model = deepcopy(topic_model)

            print(f'Current coherence score: {coherence}')
            print(f'Current best coherence score: {best_coherence}')

Running min_topic_size: 50,                     nr_topics: 8,                     top_n_words: 10


Batches:   0%|          | 0/3049 [00:00<?, ?it/s]

2023-05-15 13:41:41,736 - BERTopic - Transformed documents to Embeddings
2023-05-15 13:42:05,303 - BERTopic - Reduced dimensionality
2023-05-15 13:42:11,803 - BERTopic - Clustered reduced embeddings
2023-05-15 13:42:18,337 - BERTopic - Reduced number of topics from 846 to 8


Current coherence score: 0.178003144285564
Current best coherence score: 0.178003144285564
Running min_topic_size: 50,                     nr_topics: 8,                     top_n_words: 15


Batches:   0%|          | 0/3049 [00:00<?, ?it/s]

...

2023-05-15 14:09:18,547 - BERTopic - Transformed documents to Embeddings
2023-05-15 14:09:35,930 - BERTopic - Reduced dimensionality
2023-05-15 14:09:41,768 - BERTopic - Clustered reduced embeddings
2023-05-15 14:09:49,935 - BERTopic - Reduced number of topics from 835 to 16


Current coherence score: 0.17490914344800532
Current best coherence score: 0.19249861728314993
Running min_topic_size: 250,                     nr_topics: 16,                     top_n_words: 15


Batches:   0%|          | 0/3049 [00:00<?, ?it/s]

2023-05-15 14:11:04,595 - BERTopic - Transformed documents to Embeddings
2023-05-15 14:11:22,283 - BERTopic - Reduced dimensionality
2023-05-15 14:11:29,778 - BERTopic - Clustered reduced embeddings
2023-05-15 14:11:37,837 - BERTopic - Reduced number of topics from 834 to 16


Current coherence score: 0.17911506234836816
Current best coherence score: 0.19249861728314993


In [ ]:
print(coherence_history)
print(best_coherence)

{'50, 8, 10': 0.178003144285564, '50, 8, 15': 0.16601448356245999, '50, 12, 10': 0.17373220944808235, '50, 12, 15': 0.17005435255346724, '50, 16, 10': 0.18636712576040623, '50, 16, 15': 0.18962235073204198, '150, 8, 10': 0.17781875967274474, '150, 8, 15': 0.18476260431343694, '150, 12, 10': 0.18145078057826403, '150, 12, 15': 0.1881781306642566, '150, 16, 10': 0.18232459071560433, '150, 16, 15': 0.17801191637835465, '250, 8, 10': 0.1875604709203813, '250, 8, 15': 0.19249861728314993, '250, 12, 10': 0.17443090570715156, '250, 12, 15': 0.1847102639678944, '250, 16, 10': 0.17490914344800532, '250, 16, 15': 0.17911506234836816}
0.19249861728314993


In [ ]:
record = pd.DataFrame.from_dict(coherence_history, orient='index', columns=['value'])
record.index.name = 'min_topic_size - nr_topics - top_n_words'
record.sort_values(by = 'value', ascending = False)

,value
min_topic_size - nr_topics - top_n_words,
"250, 8, 15",0.192499
"50, 16, 15",0.189622
"150, 12, 15",0.188178
"250, 8, 10",0.187560
"50, 16, 10",0.186367
"150, 8, 15",0.184763
"250, 12, 15",0.184710
"150, 16, 10",0.182325
"150, 12, 10",0.181451


In [ ]:
best_model.save('chatgpt-topic-model')

In [ ]:
umap_model = UMAP(n_components=5, n_neighbors=15, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', prediction_data=True)

best_model = BERTopic(language="english", top_n_words=5,
                        min_topic_size=50,
                        nr_topics=15,
                        calculate_probabilities=True,
                        umap_model=umap_model,
                        hdbscan_model=hdbscan_model,
                        verbose=True)

best_model.fit(df['Tweet'].values)

Batches:   0%|          | 0/3049 [00:00<?, ?it/s]

2023-05-15 13:07:19,314 - BERTopic - Transformed documents to Embeddings
2023-05-15 13:07:37,895 - BERTopic - Reduced dimensionality
2023-05-15 13:07:43,506 - BERTopic - Clustered reduced embeddings
2023-05-15 13:07:48,775 - BERTopic - Reduced number of topics from 849 to 15


In [ ]:
# Reduce outliers using the `c-tf-idf` strategy
topics, prob = best_model.transform(df['Tweet'].values)
new_topics = best_model.reduce_outliers(df['Tweet'].values, topics, strategy="embeddings")
best_model.update_topics(df['Tweet'].values, topics=new_topics)

Batches:   0%|          | 0/3049 [00:00<?, ?it/s]

2023-05-15 13:09:12,946 - BERTopic - Reduced dimensionality
2023-05-15 13:09:13,253 - BERTopic - Predicted clusters


In [ ]:
freq = best_model.get_topic_info(); freq

,Topic,Count,Name
0,-1,6,-1_thorough_socialist_film_directlyindirectly
1,0,75623,0_chat_gpt4_like_use
2,1,4996,1_thank_congratulation_thanks_good
3,2,3976,2_use_math_answer_problem
4,3,3594,3_ask_question_asked_asking
5,4,3704,4_chat_say_dont_people
6,5,2441,5_2023_prompt_code_number
7,6,3216,6_like_human_would_world


In [ ]:
### Menentukan Nilai Coherence dari Topik
top_words = get_top_words(best_model.get_topics())
print(calculate_coherence(w2v_model, top_words))

0.19249861728314993


In [ ]:
fig = best_model.visualize_topics(); fig

# Visualisasi Akhir

In [ ]:
doc_info = topic_model.get_document_info(X_predict)
topics = doc_info['Topic'].values
top_words = get_top_words(topic_model.get_topics())

In [ ]:
df_predict['topics'] = topics
df_predict['top_words'] = df_predict['topics'].map(lambda row: top_words[row])
df_predict

,tweets,labels,topics,top_words
0,ChatGPT is another woke machine.,0,0,"[chat, gpt4, google, like, use, chatgpt4, get,..."
1,"of the Atlantic, or only near the Atla #推特账号 m...",0,0,"[chat, gpt4, google, like, use, chatgpt4, get,..."
2,This thread is saved to your Notion database....,2,0,"[chat, gpt4, google, like, use, chatgpt4, get,..."
3,Prompt AI – ChatGPT #0018,0,0,"[chat, gpt4, google, like, use, chatgpt4, get,..."
4,Just had some interesting conversations with G...,1,0,"[chat, gpt4, google, like, use, chatgpt4, get,..."
...,...,...,...,...
98754,Sorry my phones on da blink n my dyslexic mes...,2,4,"[app, code, like, game, using, apps, new, use,..."
98755,If someone were to tell a future version of C...,0,3,"[domain, business, linkedin, website, marketin..."
98756,ChatGPT can talk to your AWS infrastructure fo...,0,0,"[chat, gpt4, google, like, use, chatgpt4, get,..."
98757,"ChatGPT is not yet a human, but some humans ha...",0,13,"[human, like, asked, life, think, world, peopl..."


In [ ]:
df_predict['labels'] = df_predict['labels'].map({0: 'bad', 1: 'good', 2: 'neutral'})
df_predict.head()

,tweets,labels,topics,top_words
0,ChatGPT is another woke machine.,bad,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."
1,"of the Atlantic, or only near the Atla #推特账号 m...",bad,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."
2,This thread is saved to your Notion database....,neutral,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."
3,Prompt AI – ChatGPT #0018,bad,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."
4,Just had some interesting conversations with G...,good,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."


In [ ]:
df_predict = pd.read_csv('/content/drive/MyDrive/TK2/data_maret_final.csv')
df_predict.head()

,tweets,labels,topics,top_words
0,ChatGPT is another woke machine.,bad,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."
1,"of the Atlantic, or only near the Atla #推特账号 m...",bad,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."
2,This thread is saved to your Notion database....,neutral,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."
3,Prompt AI – ChatGPT #0018,bad,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."
4,Just had some interesting conversations with G...,good,0,"['chat', 'gpt4', 'google', 'like', 'use', 'cha..."


In [ ]:
df_plot = pd.DataFrame({'topics': df_predict.topics.unique()}).sort_values('topics').reset_index(drop=True)

for i in range(df_plot.shape[0]):
    sentiment_percent = df_predict.query(f'topics == {i}')['labels'].value_counts(normalize=True)
    df_plot.loc[i, sentiment_percent.index] = sentiment_percent.values

df_plot

df_plot

,topics,bad,good,neutral
0,0,0.519146,0.241300,0.239554
1,1,0.238681,0.556736,0.204584
2,2,0.507914,0.237896,0.254190
3,3,0.437517,0.311741,0.250742
4,4,0.442510,0.313802,0.243687
5,5,0.609657,0.169284,0.221059
6,6,0.495828,0.258023,0.246149
7,7,0.458657,0.303180,0.238163
8,8,0.436598,0.312073,0.251329
9,9,0.483298,0.283404,0.233298


In [ ]:
df_plot.to_excel('plot.xlsx', index=False)

In [ ]:
df_predict.to_csv('/content/drive/MyDrive/TK2/data_maret_final.csv', index=False)

In [ ]:
### Menentukan Nilai Coherence dari Topik
top_words = get_top_words(best_model.get_topics())
print(calculate_coherence(w2v_model, top_words))

0.1935972570500649
